# Linear Classification

In this lab you will implement parts of a linear classification model using the regularized empirical risk minimization principle. By completing this lab and analysing the code, you gain deeper understanding of these type of models, and of gradient descent.


## Problem Setting

The dataset describes diagnosing of cardiac Single Proton Emission Computed Tomography (SPECT) images. Each of the patients is classified into two categories: normal (1) and abnormal (0). The training data contains 80 SPECT images from which 22 binary features have been extracted. The goal is to predict the label for an unseen test set of 187 tomography images.

In [13]:
import urllib
import pandas as pd
import numpy as np
%load_ext autoreload
%autoreload 2

testfile = urllib.request.URLopener()
testfile.retrieve("http://archive.ics.uci.edu/ml/machine-learning-databases/spect/SPECT.train", "SPECT.train")
testfile.retrieve("http://archive.ics.uci.edu/ml/machine-learning-databases/spect/SPECT.test", "SPECT.test")

df_train = pd.read_csv('SPECT.train',header=None)
df_test = pd.read_csv('SPECT.test',header=None)

train = df_train.values
test = df_test.values

y_train = train[:,0]
X_train = train[:,1:]
y_test = test[:,0]
X_test = test[:,1:]

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Exercise 1

Analyze the function learn_reg_ERM(X,y,lambda) which for a given $n\times m$ data matrix $\textbf{X}$ and binary class label $\textbf{y}$ learns and returns a linear model $\textbf{w}$.
The binary class label has to be transformed so that its range is $\left \{-1,1 \right \}$. 
The trade-off parameter between the empirical loss and the regularizer is given by $\lambda > 0$. 
To adapt the learning rate the Barzilai-Borwein method is used.

Try to understand each step of the learning algorithm and comment each line.


In [6]:
def learn_reg_ERM(X,y,lbda):
    max_iter = 200 # even if the algorithm doesn't converge, we want to terminate it/find a solution after max_iter
    e  = 0.001 # set the step size, quantity of change between "old" and "new" model
    alpha = 1. # use inexact line search 

    w = np.random.randn(X.shape[1]); # create a random normally distributed weight vector w
    for k in np.arange(max_iter):
        h = np.dot(X,w) # apply the weight vector w to the training data (via computing the dot product) 
        l,lg = loss(h, y) # compute loss between weighted data and label and compute the corresponding loss gradient
        print ('loss: {}'.format(np.mean(l))) # print the current mean loss 
        r,rg = reg(w, lbda) # compute the L2-regularizer and the gradient of the regularizer function, regularize the model
        g = np.dot(X.T,lg) + rg 
        if (k > 0):
            alpha = alpha * (np.dot(g_old.T,g_old))/(np.dot((g_old - g).T,g_old)) 
            print('alpha: ' + str(alpha))
        w = w - alpha * g
        if (np.linalg.norm(alpha * g) < e):
            break
        g_old = g
    return w

### Exercise 2

Fill in the code for the function loss(h,y) which computes the hinge loss and its gradient. 
This function takes a given vector $\textbf{y}$ with the true labels $\in \left \{-1,1\right \}$ and a vector $\textbf{h}$ with the function values of the linear model as inputs. The function returns a vector $\textbf{l}$ with the hinge loss $\max(0, 1 − y_{i} h_{i})$ and a vector $\textbf{g}$ with the gradients of the hinge loss w.r.t $\textbf{h}$. (Note: The partial derivative of the hinge loss with respect to $\textbf{h}$  is $g_{i} = −y $ if $l_{i} > 0$, else $g_{i} = 0$)

In [2]:
def loss(h, y):
    if1 = 1 - np.multiply(h, y) # 1 - y*f(x)
    l = np.where(if1 > 0, if1, 0) #loss function
    g = - y # gradients
    return l, g

### Exercise 3

Fill in the code for the function reg(w,lambda) which computes the $\mathcal{L}_2$-regularizer and the gradient of the regularizer function at point $\textbf{w}$. 


$$r = \frac{\lambda}{2} \textbf{w}^{T}\textbf{w}$$

$$g = \lambda \textbf{w}$$

In [3]:
def reg(w, lbda):
    r = (lbda/2)*np.dot(w.T, w)
    g = lbda*w
    return r, g

### Exercise 4

Fill in the code for the function predict(w,x) which predicts the class label $y$ for a data point $\textbf{x}$ or a matrix $X$ of data points (row-wise) for a previously trained linear model $\textbf{w}$. If there is only a data point given, the function is supposed to return a scalar value. If a matrix is given a vector of predictions is supposed to be returned.

In [4]:
def predict(w, X):
    preds = 2*(np.dot(X, w) > 0) - 1
    return preds

### Exercise 5

#### 5.1 
Train a linear model on the training data and classify all 187 test instances afterwards using the function predict. 
Please note that the given class labels are in the range $\left \{0,1 \right \}$, however the learning algorithm expects a label in the range of $\left \{-1,1 \right \}$. Then, compute the accuracy of your trained linear model on both the training and the test data. 

In [14]:
# convert the labels
from sklearn.metrics import accuracy_score

y_train_n = np.array([-1 if y_i==0 else y_i for y_i in y_train])
y_pred_train  = predict(learn_reg_ERM(X_train, y_train_n, 0.5), X_train)

print(f"y-predicted:", y_pred_train)
print(f"y-Train:", y_train)
print(f"Accuracy train:", accuracy_score(y_train_n, y_pred_train))

y_test_n = np.array([-1 if y_i==0 else y_i for y_i in y_test])
y_pred_test  = predict(learn_reg_ERM(X_test, y_test_n, 0.5), X_test)
print(f"Accuracy train:{accuracy_score(y_test_n, y_pred_test)}")

loss: 2.2495144509007745
loss: 10.880499515806449
alpha: 2.0
loss: 21.675
alpha: 2.0
y-predicted: [ 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1 -1 -1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
 -1  1  1 -1 -1  1  1  1  1  1 -1 -1  1 -1  1 -1 -1 -1  1 -1  1 -1  1  1
  1  1 -1  1  1  1  1  1]
y-Train: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0]
Accuracy train: 0.625
loss: 1.5015545529652723
loss: 15.360509491620183
alpha: 2.0
loss: 30.62566844919786
alpha: 2.0
Accuracy train:0.9358288770053476


#### 5.2
Compare the accuracy of the linear model with the accuracy of a random forest and a decision tree on the training and test data set.

In [15]:
from sklearn.ensemble import RandomForestClassifier
# train a random forest
rndm_clf = RandomForestClassifier()
rndm_clf.fit(X_train, y_train)
print(rndm_clf.score(X_test, y_test))

from sklearn.tree import DecisionTreeClassifier

dcsn_clf = DecisionTreeClassifier()
dcsn_clf.fit(X_train, y_train)
print(dcsn_clf.score(X_test, y_test))

0.7700534759358288
0.6951871657754011
